In [1]:
from algom.utils.data_object import dataObject
from src.trade.get_trades import get_business_rules_response

<br><br>

### Get input data


In [2]:
data = dataObject("""
select
ticker_time_sec,
ticker_time,
ticker,
`interval`,
exchange,
week,
month,
quarter,
year,
close,
high,
low,
open,
volume_base,
volume,
AROR_6,
AROR_12,
AROR_24,
LAG1_AROR_6,
LAG2_AROR_6,
LAG3_AROR_6,
LAG1_AROR_12,
LAG2_AROR_12,
LAG3_AROR_12,
LAG3_AROR_24,
Bollinger_pct_b_48,
Bollinger_pct_b_96,
Bollinger_pct_b_240,
lag(Bollinger_pct_b_48, 3) over (order by ticker_time_sec) AS LAG3_Bollinger_pct_b_48,
lag(Bollinger_pct_b_96, 3) over (order by ticker_time_sec) AS LAG3_Bollinger_pct_b_96,
from `algom-trading.train_features.features_BTC_USDT_hour_i05_20190101`
where year = 2019
order by ticker_time
""")

RUNNING: Querying SQL script.


Downloading: 100%|██████████| 8760/8760 [00:03<00:00, 2258.48rows/s]

SUCCESS: Loaded SQL query.


In [3]:
data.df.head()

,ticker_time_sec,ticker_time,ticker,interval,exchange,week,month,quarter,year,close,...,LAG3_AROR_6,LAG1_AROR_12,LAG2_AROR_12,LAG3_AROR_12,LAG3_AROR_24,Bollinger_pct_b_48,Bollinger_pct_b_96,Bollinger_pct_b_240,LAG3_Bollinger_pct_b_48,LAG3_Bollinger_pct_b_96
0,1546300800,2019-01-01 00:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2019,3700.31,...,-0.003625,-0.000505,-0.001901,-0.002325,-0.001386,0.143179,0.358759,0.305245,NaN,NaN
1,1546304400,2019-01-01 01:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2019,3689.69,...,-0.002234,-0.000701,-0.000505,-0.001901,-0.001209,0.076420,0.318606,0.287154,NaN,NaN
2,1546308000,2019-01-01 02:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2019,3690.00,...,-0.001497,-0.000955,-0.000701,-0.000505,-0.001099,0.091346,0.312133,0.289160,NaN,NaN
3,1546311600,2019-01-01 03:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2019,3693.13,...,-0.001846,-0.001778,-0.000955,-0.000701,-0.001057,0.120850,0.313055,0.296102,0.143179,0.358759
4,1546315200,2019-01-01 04:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2019,3692.71,...,0.000100,-0.001328,-0.001778,-0.000955,-0.000819,0.134330,0.301597,0.296472,0.076420,0.318606


<br><br><br>

### Apply backtest


In [4]:
from src.trade import backtest

In [5]:
bt = backtest.backtestTrades(
    df=data.df,
    business_rule_functions='src.business_rules.rules_BTCUSDT_hour_i02'
)

In [6]:
#  02.2 - % Gain: -63.5% vs -72.6% market 2019
#  02.3 - % Gain: 4.0% vs -72.6% market

#  02.1 - % Gain: 62.2% vs 94.4% market 2019
#  02.2 - % Gain: 71.5% vs 94.4% market 2019
#  02.3 - % Gain: 71.5% vs 94.4% market 2019

bt.run_backtest()

BACKTEST: Getting backtest metadata
BACKTEST: Applying trade signals based on business rules module.
BACKTEST: Calculating performance metrics based on business rules module.

        ---------- BACKTEST PERFORMANCE SUMMARY ----------
        - % Gain: 3.1% vs 94.4% market
        - Income: $310.0
        - Account Value: $10310.0
        - Account Cash:  $1392.0
        - Trades: 631 of 940 attempts (67.0% execution rate)
        - Account Min: $8613
        - Account Max: $13629
        
BACKTEST: Results will not be loaded to BigQuery.


In [7]:
!pwd

/home/jovyan/algomosaic/notebooks/algom_trading_v001_dev/binance_hour/04_backtest


In [8]:
bt.backtest_output.to_csv('backtest_output.csv')

In [9]:
# bt.backtest_summary
# list(bt.df)


<br><br>

### Apply trade responses


In [5]:
df = get_business_rules_response(
    data=data.df, 
    function='trade', 
    module='src.business_rules.trade_business_rules_BTCUSDT', 
    column_name=None, 
    transpose=True
)

In [6]:
df.head()

,ticker_time_sec,ticker_time,ticker,interval,exchange,week,month,quarter,year,close,...,low,open,volume_base,volume,AROR_12,AROR_24,response,trade,amount,commission
0,1514764800,2018-01-01 00:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2018,13529.01,...,13400.01,13716.36,426.78,5769896.49,0.002991,0.002146,"{'trade': 'SELL', 'amount': 0.36958, 'commissi...",SELL,0.36958,0.0
1,1514768400,2018-01-01 01:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2018,13203.06,...,13155.38,13529.01,359.65,4832291.65,0.000285,0.000783,None,None,NaN,NaN
2,1514772000,2018-01-01 02:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2018,13330.18,...,13200.00,13203.06,405.95,5402610.44,-0.000048,-0.000054,"{'trade': 'SELL', 'amount': 0.37509, 'commissi...",SELL,0.37509,0.0
3,1514775600,2018-01-01 03:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2018,13410.03,...,13290.00,13330.18,406.22,5470651.84,0.001235,0.001416,None,None,NaN,NaN
4,1514779200,2018-01-01 04:00:00+00:00,BTC-USDT,hour,binance,1,1,1,2018,13601.01,...,13322.15,13410.03,326.49,4394870.97,-0.001427,0.002126,"{'trade': 'BUY', 'amount': 0.07352, 'commissio...",BUY,0.07352,0.0


In [10]:
# len(df[df.trade=='BUY'])
df[df.trade=='BUY'].sample(2)

,ticker_time_sec,ticker_time,ticker,interval,exchange,week,month,quarter,year,close,...,low,open,volume_base,volume,AROR_12,AROR_24,response,trade,amount,commission
1159,1518937200,2018-02-18 07:00:00+00:00,BTC-USDT,hour,binance,7,2,1,2018,10589.76,...,10391.75,10425.00,2683.98,28196764.28,-0.001019,-0.000540,"{'trade': 'BUY', 'amount': 0.09443, 'commissio...",BUY,0.09443,0.0
8307,1544670000,2018-12-13 03:00:00+00:00,BTC-USDT,hour,binance,50,12,4,2018,3405.78,...,3393.00,3403.02,859.48,2921465.28,-0.001323,0.000494,"{'trade': 'BUY', 'amount': 0.29362, 'commissio...",BUY,0.29362,0.0


In [9]:
len(df[df.trade=='SELL'])
df[df.trade=='SELL'].sample(2)

,ticker_time_sec,ticker_time,ticker,interval,exchange,week,month,quarter,year,close,...,low,open,volume_base,volume,AROR_12,AROR_24,response,trade,amount,commission
5388,1534161600,2018-08-13 12:00:00+00:00,BTC-USDT,hour,binance,33,8,3,2018,6443.96,...,6435.00,6441.34,1958.15,12633034.79,0.001592,0.001207,"{'trade': 'SELL', 'amount': 0.77592, 'commissi...",SELL,0.77592,0.0
3181,1526216400,2018-05-13 13:00:00+00:00,BTC-USDT,hour,binance,19,5,2,2018,8551.82,...,8505.99,8571.93,672.28,5742406.63,0.001285,0.000194,"{'trade': 'SELL', 'amount': 0.58467, 'commissi...",SELL,0.58467,0.0
